In [2]:
import os
import string
import annoy
import codecs
import pickle
import json

from pymorphy3 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec

import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split

Предобработаем ответы mail.ru из файла: к каждому вопросу присоединим 1 ответ и запишем в файл на будущее. Это позволит нам сэкономить время и ресурсы при дальнейшем препроцессинге текста

In [2]:
question = None
written = False

#Мы идем по всем записям, берем первую строку как вопрос
# и после знака --- находим ответ
with codecs.open("data/prepared_answers.txt","w", "utf-8") as fout:
    with codecs.open("data/Otvety.txt", "r", "utf-8") as fin:
        for line in tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            elif not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            elif not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

Теперь нам нужно предобработать текст, чтобы обучить word2vec и получить эмбеддинги. Удаляем знаки препинания и делаем лемматизацию

In [7]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [10]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with codecs.open("data/Otvety.txt", "r", "utf-8") as fin:
    for line in tqdm(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

0it [00:00, ?it/s]

In [11]:
# Обучим модель word2vec на наших вопросах
sentences = [i for i in sentences if len(i) > 2]
model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)
model.save("w2v_model")

Теперь нам нужно сложить в индекс все вопросы. Используем библиотеку annoy. Проходимся по всем ответам, считаем, что вектор предложения - сумма word2vecов слов, которые входят в него (конечно же усредненная)

In [21]:
index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with codecs.open("data/prepared_answers.txt", "r", "utf-8") as f:
    for line in tqdm(f):
        n_w2v = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector = np.zeros(100)
        for word in question:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)
            
        counter += 1

index.build(10)
index.save('speaker.ann')

0it [00:00, ?it/s]

True

Теперь остается реализовать метод, который получит на вход вопрос и найдет ответ к нему!
Мы препроцессим вопрос, находим ближайший вопрос и выбираем ответ на ближайший вопрос.

In [7]:
def find_answer(vector):
    # preprocessed_question = preprocess_txt(question)
    # n_w2v = 0
    # vector = np.zeros(100)
    # for word in preprocessed_question:
    #     if word in model.wv:
    #         vector += model.wv[word]
    #         n_w2v += 1
    # if n_w2v > 0:
    #     vector = vector / n_w2v
    answer_index = index.get_nns_by_vector(vector, 1)
    return index_map[answer_index[0]]

Теперь останем из данных о продуктах название продукта и его описание.

In [3]:
products = pd.read_csv('data/ProductsDataset.csv')
display(products)

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...
...,...,...,...,...,...,...,...
35543,Юбка,Юбка Белая по.Турция фирма adL,5b5f181c62e1c6616a7f6472,9.0,904,"{'zhenskaya_odezhda_platya_yubki_tip': 'Юбки',...",http://cache3.youla.io/files/images/360_360/5b...
35544,Новый твидовый пиджак,Новый с бирками пиджак размер S в стиле Coco C...,5bd6c8b29e94ba033d31f8d0,9.0,908,"{'brand_zhenskii': 'Chanel', 'zhenskaya_odezhd...",http://cache3.youla.io/files/images/360_360/5b...
35545,Женская зимняя куртка,Женская зимняя спортивная куртка фирмы Rossiqn...,5bd6c8bc074b3e1c056f69b2,9.0,903,"{'zhenskaya_odezhda_razmer': '48-50 (XL)', 'zh...",http://cache3.youla.io/files/images/360_360/5b...
35546,Новая золотая ветровка,Женская ветровка размер 44-46. Цвет приглушённ...,5bd6c8fb2138bbc55745362c,9.0,903,"{'zhenskaya_odezhda_razmer': '44-46 (М)', 'zhe...",http://cache3.youla.io/files/images/360_360/5b...


In [4]:
products_title = list(products['title'])
products_descrirption = list(products['descrirption'])

На мой взгляд достаточно было проверять соттветствие вопроса наванию продукта или его описанию, но задание гласит обучить классификаор на основе векторов. 

Считаю, что в данном случае должноучитываться полное соответствие так что модель обучаем на полное соответсвие векторов вопроса и название или его описание продукта.

Находим векторы для названия и описания продуктов. Добавляем их df

In [12]:
p =[]
for elem in products_title:
    if type(elem) is str:
        elem = preprocess_txt(elem)
        n_w2v = 0
        vector = np.zeros(100)
        for word in elem:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
    else:
        vector = np.zeros(100)
    p.append(vector)
        
    
products_title = p

p =[]
for elem in products_descrirption:
    if type(elem) is str:
        elem = preprocess_txt(elem)
        n_w2v = 0
        vector = np.zeros(100)
        for word in elem:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
    else:
        vector = np.zeros(100)
    p.append(vector)
    
products_descrirption = p


In [50]:
title_df = pd.DataFrame(products_title, columns=[f'elem_{elem}' for elem in range(100)])
title_df['label']=True
title_df.head()

,elem_0,elem_1,elem_2,elem_3,elem_4,elem_5,elem_6,elem_7,elem_8,elem_9,...,elem_91,elem_92,elem_93,elem_94,elem_95,elem_96,elem_97,elem_98,elem_99,label
0,-0.969692,1.184397,0.321013,0.217386,1.022203,-0.443842,-0.596730,1.621000,-0.524536,-0.223706,...,-0.719325,0.639190,-1.587708,1.411872,-0.253069,1.933632,-0.591978,0.433518,0.383290,True
1,-0.046701,0.060006,0.032042,-0.003643,-0.013172,-0.026308,-0.043181,0.096244,-0.013404,-0.000433,...,-0.007951,0.027835,-0.040610,0.077723,0.024170,0.068311,-0.044373,0.003868,0.014565,True
2,-1.506026,0.623752,0.447523,0.450710,0.017114,0.194888,-1.022076,1.098813,-0.841371,-0.048318,...,-0.469947,0.171516,-0.631438,0.851952,0.133723,1.222949,-0.587822,0.701460,-0.632027,True
3,-0.596135,0.985707,1.164543,0.051252,0.890716,-0.980987,0.081230,1.327935,-0.970258,-0.683786,...,-0.675924,0.375552,-1.286004,1.131824,-0.626015,1.744366,-0.551295,0.684208,0.451767,True
4,-0.333473,0.261219,0.104462,0.114841,-0.067490,0.032081,-0.354605,0.482128,-0.197891,-0.083412,...,-0.116895,0.131349,-0.260268,0.219402,0.051401,0.359988,-0.191232,0.212605,-0.097482,True


In [49]:
description_df = pd.DataFrame(products_descrirption, columns=[f'elem_{elem}' for elem in range(100)])
description_df['label']=True
description_df.head()

,elem_0,elem_1,elem_2,elem_3,elem_4,elem_5,elem_6,elem_7,elem_8,elem_9,...,elem_91,elem_92,elem_93,elem_94,elem_95,elem_96,elem_97,elem_98,elem_99,label
0,-0.422273,0.895296,0.253122,-0.086418,-0.021820,-0.965250,-0.635853,1.336728,-0.734654,-0.409430,...,-0.869746,-0.830293,-0.770820,0.982751,-0.364856,2.342862,-0.474167,0.398306,0.182188,True
1,-0.463555,0.711118,0.543835,0.279032,0.404274,-1.287037,-0.006462,1.418218,-1.049128,-0.088385,...,-0.570652,-0.362557,-0.950282,0.923482,0.520834,1.077326,-1.131352,0.208201,0.030356,True
2,-0.613022,0.927975,0.828570,-0.015077,0.508361,-1.489316,-0.148693,1.276741,-1.099884,-0.582578,...,-0.991274,-0.367761,-0.859645,0.937124,0.125580,1.076449,-1.202528,0.329439,0.196596,True
3,-0.625045,1.248635,1.413205,0.348568,-0.229934,-1.347809,0.079768,1.363074,-1.582813,-1.089018,...,-1.669296,0.264788,-1.323691,0.841481,-0.605140,1.938368,-1.093752,1.265731,-0.447682,True
4,0.326546,1.084318,-0.042386,0.308835,0.358523,-1.546638,0.258828,1.515495,-0.659027,-0.331434,...,-0.514432,-0.580303,-1.168020,0.474493,0.065082,1.021905,-0.943917,0.272617,0.563324,True


In [13]:
products['title_v'] = products_title
products['descrirption_v'] = products_descrirption
products.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,title_v,descrirption_v
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[-0.8910114765167236, 1.3787912726402283, 0.54...","[-0.41696231785463167, 0.9401321589946747, 0.0..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[-0.030345773324370384, 0.04688362032175064, 0...","[-0.3726481427050506, 0.8394228785764426, 0.54..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[-1.343453049659729, 0.471617192029953, 0.4767...","[-0.6581696506451677, 1.0949027720424864, 0.78..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[-0.4463424930969874, 1.2275214890638988, 1.21...","[-0.4239525869488716, 1.2775271534919739, 0.94..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[-0.36461570858955383, 0.18958771228790283, 0....","[0.2151714459889465, 1.2760183562835057, 0.016..."


In [14]:
products['title_lower'] = products['title'].str.lower()
products['descrirption_lower'] = products['descrirption'].str.lower()

Сохраняем в файл для дальнейшего использования.

In [46]:
products.to_csv('data/vector_products.csv')

In [51]:
products.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,title_v,descrirption_v,title_lower,descrirption_lower
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[-0.9696923196315765, 1.1843972206115723, 0.32...","[-0.42227291464805605, 0.8952961802482605, 0.2...",юбка детская orby,"новая, не носили ни разу. в реале красивей чем..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[-0.04670074209570885, 0.060005947947502136, 0...","[-0.4635552189235265, 0.7111176467345407, 0.54...",ботильоны,"новые,привезены из чехии ,указан размер 40,но ..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[-1.506026029586792, 0.6237518787384033, 0.447...","[-0.6130216005603196, 0.9279745364113262, 0.82...",брюки,размер 40-42. брюки почти новые - не знаю как ...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[-0.5961351891358694, 0.9857067167758942, 1.16...","[-0.6250451952219009, 1.2486350536346436, 1.41...",продам детские шапки,"продам шапки,кажда 200р.розовая и белая проданны."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[-0.3334726095199585, 0.26121893525123596, 0.1...","[0.32654620955387753, 1.0843178675406508, -0.0...",блузка,"темно-синяя, 42 размер,состояние отличное,как ..."


вычисляем вектора вопросов.

In [13]:
questions =[]
with codecs.open("data/prepared_answers.txt", "r", "utf-8") as f:
    for line in tqdm(f):
        elem = preprocess_txt(line.split("\t")[0])
        n_w2v = 0
        vector = np.zeros(100)
        for word in elem:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        questions.append(vector)

len(questions)

0it [00:00, ?it/s]

1163342

In [ ]:
question_df = pd.DataFrame(questions, columns=[f'elem_{elem}' for elem in range(100)])


In [ ]:
question_df['question'] = questions

In [52]:
question_df.head()

,elem_0,elem_1,elem_2,elem_3,elem_4,elem_5,elem_6,elem_7,elem_8,elem_9,...,elem_93,elem_94,elem_95,elem_96,elem_97,elem_98,elem_99,question,question_text,label
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",\tвопрос о тдв)) давно и хорошо отдыхаем)) лич...,False
1,-0.582070,1.270758,0.253506,0.288584,0.104033,-0.834501,-0.667006,1.903806,-0.883800,-0.203698,...,-0.339334,1.452323,0.123868,2.143351,-0.695643,0.822582,-0.897363,"[-0.5820701587945223, 1.2707577347755432, 0.25...",как парни относятся к цветным линзам? если у д...,False
2,-0.231507,0.297988,2.292807,1.449215,0.889021,-2.529247,1.833910,1.682538,-1.999187,-0.651845,...,-0.167768,1.451499,0.183007,1.178739,-2.136928,0.654762,0.868864,"[-0.231506872177124, 0.2979876041412354, 2.292...","что делать, сегодня нашёл 2 миллиона рублей? ....",False
3,-0.271320,0.240729,-0.005571,0.122147,0.155937,-0.528363,0.101260,0.338586,-0.335280,-0.143837,...,-0.152649,0.182906,0.135302,0.396139,-0.230727,0.073704,0.063777,"[-0.27131985072046516, 0.2407290842034854, -0....",эбу в двенашке называется итэлма что за эбу? ....,False
4,0.079112,0.928494,0.169909,-1.069200,0.615266,-1.709731,-0.797414,2.451779,0.639208,-1.194929,...,-0.874296,-0.077365,0.362335,0.239875,-0.373200,-0.173734,0.979038,"[0.07911221473477781, 0.9284941032528877, 0.16...",академия вампиров. сколько на даный момент час...,False


Добавляем в Датафрейм текст запроса

In [20]:
questions_text =[]
with codecs.open("data/prepared_answers.txt", "r", "utf-8") as f:
    for line in tqdm(f):
        questions_text.append(line.lower())
questions_text

0it [00:00, ?it/s]

['\tвопрос о тдв)) давно и хорошо отдыхаем)) лично вам здесь кого советовали завести?)) . \n',
 'как парни относятся к цветным линзам? если у девушки то зеленые глаза, то голубые...)) .\tменя вобще прикалывает эта тема :). \n',
 'что делать, сегодня нашёл 2 миллиона рублей? .\tесли это "счастье " действительно на вас свалилось, лучше пойти в милицию и заявить о находке. такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. можно нарваться на бабушку конечно, которая хотела помоч внуку с покупкой квартиры, а можно на бандитов, которые будут с вами разговаривать иначе чем бабушка с милицией. выбор за вами, есть еще конечно шанс, что это подарок с выше за котрый с вас никто не спросит, тогда лучше отдать хотябы 500 на благотворительность. дабы не спугнуть удачу!. \n',
 'эбу в двенашке называется итэлма что за эбу? .\tэбу — электронный блок управления двигателем автомобиля, его другое название — контроллер. он при

In [ ]:
question_df['question_text'] = questions_text

Создаем Датафрейм для обучения модели

In [53]:
class_df = question_df.drop(['question', 'question_text'], axis=1)
class_df.head()

,elem_0,elem_1,elem_2,elem_3,elem_4,elem_5,elem_6,elem_7,elem_8,elem_9,...,elem_91,elem_92,elem_93,elem_94,elem_95,elem_96,elem_97,elem_98,elem_99,label
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False
1,-0.582070,1.270758,0.253506,0.288584,0.104033,-0.834501,-0.667006,1.903806,-0.883800,-0.203698,...,-0.995599,-0.234644,-0.339334,1.452323,0.123868,2.143351,-0.695643,0.822582,-0.897363,False
2,-0.231507,0.297988,2.292807,1.449215,0.889021,-2.529247,1.833910,1.682538,-1.999187,-0.651845,...,-1.306614,-0.442983,-0.167768,1.451499,0.183007,1.178739,-2.136928,0.654762,0.868864,False
3,-0.271320,0.240729,-0.005571,0.122147,0.155937,-0.528363,0.101260,0.338586,-0.335280,-0.143837,...,0.287905,0.142711,-0.152649,0.182906,0.135302,0.396139,-0.230727,0.073704,0.063777,False
4,0.079112,0.928494,0.169909,-1.069200,0.615266,-1.709731,-0.797414,2.451779,0.639208,-1.194929,...,-0.579284,0.503418,-0.874296,-0.077365,0.362335,0.239875,-0.373200,-0.173734,0.979038,False


In [68]:
class_df = pd.concat([class_df, title_df, description_df])
len(class_df)

1305534

Сохраняем датафрейм чтобы потом не пересчитывать

In [59]:
class_df.to_csv('data/for_model.csv')

Разбиваем выборку

In [69]:
X = class_df.drop(['label'], axis=1)
y = class_df['label']

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.3)

Обучаем логистическую регрессию

In [72]:
log_model = linear_model.LogisticRegression(max_iter = 1000, random_state=42)

log_model.fit(X_train, y_train)

y_test_pred = log_model.predict(X_test)
print(f'f1_score на тестовом наборе: {metrics.f1_score(y_test, y_test_pred):.2f}')

f1_score на тестовом наборе: 0.78


Сохраняем в файл

In [74]:
with open('log_model.pkl', 'wb') as file:
    pickle.dump(log_model, file, protocol=pickle.HIGHEST_PROTOCOL)

Пишем функцию обработки продуктового запроса.

In [15]:
product_index = annoy.AnnoyIndex(100 ,'angular')

product_index_map = {}
counter = 0

for counter in range(len(products.index)):
    product_index_map[counter] = f'{products.iloc[counter].title} ({products.iloc[counter].product_id})'
    product_index.add_item(counter, products.iloc[counter]['title_v'])
    product_index_map[len(products.index) + counter] = f'{products.iloc[counter].title} ({products.iloc[counter].product_id})'
    product_index.add_item(len(products.index) + counter, products.iloc[counter]['descrirption_v'])

product_index.build(10)
product_index.save('product.ann')

True

In [16]:
def find_product(vector):
    answer_index = product_index.get_nns_by_vector(vector, 1)
    return product_index_map[answer_index[0]]


Пишем функцию обработки запроса.

In [17]:
def get_answer(question):
    question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    direction = log_model.predict(vector.reshape(1, -1))
    if direction:
        return find_product(vector)
    else:
        return find_answer(vector)
        
                
            
    

Записываем словари в фалы для передачи в файл бота.

In [22]:
with open('index_map.json', 'w') as f:
    json.dump(index_map, f)
    

In [19]:
with open('product_index_map.json', 'w') as f:
    json.dump(product_index_map, f)

Просто для интереса импортируем через pickle

In [23]:
with open('index_map.pkl', 'wb') as file:
    pickle.dump(index_map, file, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
with open('product_index_map.pkl', 'wb') as file:
    pickle.dump(product_index_map, file, protocol=pickle.HIGHEST_PROTOCOL)

Тестируем:

In [94]:
get_answer('Юбка детская ORBY')

c:\Projects\DS\lessons\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


'Юбка детская ORBY (58e3cfe6132ca50e053f5f82)'

In [95]:
get_answer('Где ключи от танка')

c:\Projects\DS\lessons\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


'Можно обратится к дилеру ТАЗа, у них есть программы для идентификации своей продукции. \n'

In [96]:
get_answer('бла бла бла')

c:\Projects\DS\lessons\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


'так ты оригинал.. \n'

In [97]:
get_answer('Юбка Белая по.Турция фирма adL')

c:\Projects\DS\lessons\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


'Юбка (5b5f181c62e1c6616a7f6472)'